In [1]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

import numpy as np




In [7]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.0.1-cp37-none-any.whl size=10032 sha256=444d9a35b93b71a507c7b5b38d9e91bbe29f2c018b903b7e2ce4e7994653e635
  Stored in directory: /Users/ieunpyo/Library/Caches/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


## Colab Setting

In [23]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [24]:
work_dir = '/content/drive/work/gimhae_fire/'


## Data Loading

In [20]:
Train_X = np.load('/Users/ieunpyo/PycharmProjects/Kaggle/gimhae_fire/Train_X.npy',allow_pickle=True)
Test_X = np.load('/Users/ieunpyo/PycharmProjects/Kaggle/gimhae_fire/Test_X.npy',allow_pickle=True)
Validation_X = np.load('/Users/ieunpyo/PycharmProjects/Kaggle/gimhae_fire/Validation_X.npy',allow_pickle=True)

Train_y = np.load('/Users/ieunpyo/PycharmProjects/Kaggle/gimhae_fire/Train_y.npy',allow_pickle=True)
Validation_y = np.load('/Users/ieunpyo/PycharmProjects/Kaggle/gimhae_fire/Validation_y.npy',allow_pickle=True)



In [22]:
Train_X.shape

(59199, 317)

# HYPERPARAMETER TUNING

## initial search

#### n_estimators : Number of trees in random forest
#### max_features : The number of features to consider when looking for the best split
#### max_depth : Maximum number of levels in tree
#### min_samples_split : Minimum number of samples required to split a node
#### min_samples_leaf : Minimum number of samples required at each leaf node

## second search
##### hyperparameter 완전 결정

## predict on test_X & submission
##### 정해진 hyperparameter로 N번 fitting 반복 predict -> f-score 최고점된 model로 제출

In [30]:
import datetime

In [31]:
now = datetime.datetime.now().strftime('%H:%M:%S')

In [33]:
'a' +now

'a17:56:26'

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# n_estimators : Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 50, num = 10)]

# max_features : The number of features to consider when looking for the best split
max_features = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# min_samples_split : Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# min_samples_leaf : Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 5, 10]


# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}



In [8]:
#Import libraries
import pandas as pd
import numpy as np
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [9]:
#Bayesian optimization
def bayesian_optimization(dataset, function, parameters):
   X_train, y_train, X_test, y_test = dataset
   n_iterations = 5
   gp_params = {"alpha": 1e-4}

   BO = BayesianOptimization(function, parameters)
   BO.maximize(n_iter=n_iterations, **gp_params)

   return BO.max

In [11]:
def rfc_optimization(cv_splits):
    def function(n_estimators, max_depth, min_samples_split):
        return cross_val_score(
               RandomForestClassifier(
                   n_estimators=int(max(n_estimators,0)),                                                               
                   max_depth=int(max(max_depth,1)),
                   min_samples_split=int(max(min_samples_split,2)), 
                   n_jobs=-1, 
                   random_state=42,   
                   class_weight="balanced"),  
               X=X_train, 
               y=y_train, 
               cv=cv_splits,
               scoring="roc_auc",
               n_jobs=-1).mean()

    parameters = {"n_estimators": (10, 1000),
                  "max_depth": (20, 150),
                  "min_samples_split": (2, 10)}
    
    return function, parameters

In [12]:
#Train model
def train(X_train, y_train, X_test, y_test, function, parameters):
    dataset = (X_train, y_train, X_test, y_test)
    cv_splits = 4
    
    best_solution = bayesian_optimization(dataset, function, parameters)      
    params = best_solution["params"]

    model = RandomForestClassifier(
             n_estimators=int(max(params["n_estimators"], 0)),
             max_depth=int(max(params["max_depth"], 1)),
             min_samples_split=int(max(params["min_samples_split"], 2)), 
             n_jobs=-1, 
             random_state=42,   
             class_weight="balanced")

    model.fit(X_train, y_train)
    
    return model